# Przygotowanie danych

## Importy

In [1]:
import os
import re
import ast
import requests
import datetime

import pandas as pd
import numpy as np

from tqdm import tqdm
from uuid import uuid4
from sqlalchemy import create_engine

## Stałe

In [2]:
MKDIR_DATA = 'data/'

MOVIES_CSV =  os.path.join(MKDIR_DATA,'movies_metadata.csv')
RATINGS_CSV = os.path.join(MKDIR_DATA,'ratings.csv')
LINKS_CSV = os.path.join(MKDIR_DATA, 'links.csv')

DB_HOST = '127.0.0.1'
DB_NAME = 'webfilm'
DB_USER = 'webfilm'
DB_PASS = 'nhy6&UJM'

## Przygotowanie polączenia z bazą

In [3]:
engine = create_engine(f'mysql+pymysql://{DB_USER}:{DB_PASS}@{DB_HOST}/{DB_NAME}')

## Utworzenie folderów

In [4]:
list_folders = [
    MKDIR_DATA,
]
for x in list_folders:
    if not os.path.exists(x):
        os.makedirs(x)

## Ustawienia środowiska

In [5]:
PANDAS_MAX_ROW = 10

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', PANDAS_MAX_ROW)

## Pobranie danych

In [6]:
!kaggle datasets download -p $MKDIR_DATA --unzip rounakbanik/the-movies-dataset

file = open(MOVIES_CSV, 'r')
wiersz = 1
usunac_nowy_wiersz = [19763, 29572, 35671]
tekst = ''
for line in tqdm(file):
    if wiersz in usunac_nowy_wiersz:
        line = line.strip()

    wiersz = wiersz + 1
    tekst += line
file.close()

file = open(MOVIES_CSV, 'w')
file.write(tekst)
file.close()

100%|████████████████████████████████████████| 228M/228M [00:07<00:00, 27.7MB/s]
100%|████████████████████████████████████████| 228M/228M [00:07<00:00, 29.9MB/s]


45573it [00:00, 152518.62it/s]


## Wczytanie danych
Wczytanie i podzielenie danych

### Movies

In [7]:
movies = pd.read_sql('SELECT id, title FROM movies_movie', engine)
movies

,id,title
0,55245,!Women Art Revolution
1,41371,#1 Cheerleader Camp
2,301325,#Horror
3,267752,#chicagoGirl
4,143747,"$1,000 on the Black"
...,...,...
45428,388182,هیچ کجا هیچ کس
45429,120249,‘Rameau’s Nephew’ by Diderot (Thanx to Dennis ...
45430,14670,’Round Midnight
45431,109380,…And the Fifth Horseman Is Fear


### Links

In [8]:
links = pd.read_csv(LINKS_CSV, keep_default_na=False, low_memory=True)
links['tmdbId'] = pd.to_numeric(links.tmdbId, downcast='integer')
links.head()

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


### Ratings

In [9]:
def process_rating_chunk(ratings):
    ratings = ratings.merge(links, left_on='movieId', right_on='movieId')
    ratings = ratings.rename(columns={
        'userId': 'user_id',
        'tmdbId': 'movie_id',
        'timestamp': 'date',
    })
    ratings = ratings.sort_values(['date'])
    ratings['date'] = pd.to_datetime(ratings['date'], unit='s')
    
    ratings = ratings.drop('movieId', axis=1)
    ratings = ratings.drop('imdbId', axis=1)
    ratings['rating'] = ratings.rating.apply(lambda x: x * 2)

    return ratings

In [10]:
ratings = pd.read_csv(RATINGS_CSV, keep_default_na=False, low_memory=True)

ratings = process_rating_chunk(ratings)
ratings = ratings[ratings['movie_id'].isin(movies['id'])]
ratings = ratings.drop_duplicates(subset=['user_id', 'movie_id', ], keep='last')
ratings

,user_id,rating,date,movie_id
20925665,38150,8.0,1995-01-09 11:46:44,1600.0
12937605,44717,6.0,1995-01-09 11:46:49,8012.0
4842674,44717,6.0,1995-01-09 11:46:49,623.0
3870682,44717,10.0,1995-01-09 11:46:49,807.0
6295319,187396,10.0,1996-01-29 00:00:00,9598.0
...,...,...,...,...
11605097,85434,4.0,2017-08-04 06:38:59,13342.0
23907954,85434,6.0,2017-08-04 06:39:06,10140.0
19982064,85434,8.0,2017-08-04 06:41:23,19186.0
20803205,199634,6.0,2017-08-04 06:53:43,1724.0


In [11]:
users = pd.DataFrame({
    'id': ratings['user_id'],
    'password': '',
    'name': '',
    'surname': '',
    'gender': 0,
    'active_status': 2,
    'role_id': 1,
}).drop_duplicates('id').sort_values('id')
users.head()

,id,password,name,surname,gender,active_status,role_id
217591,1,,,,0,2,1
998951,2,,,,0,2,1
1500966,3,,,,0,2,1
2259410,4,,,,0,2,1
2714139,5,,,,0,2,1


In [12]:
ratings.to_csv('ratings.csv', index=False)

In [13]:
list_tabels = [
    { 'nazwa_tabeli': 'users_user', 'data': users, },
    { 'nazwa_tabeli': 'evaluations_rating', 'data': ratings, },
]

In [14]:
for x in tqdm(list_tabels):
    x['data'].to_sql(f'{x["nazwa_tabeli"]}', con=engine, if_exists='append', index=False, method='multi', chunksize=1000)

100%|███████████████████████████████████████| 2/2 [10:56:45<00:00, 19702.78s/it]
